# fotmob

In [3]:
import re
import urllib.parse
from logging import getLevelName, getLogger
from typing import Optional, Union

import requests
from cachecontrol import CacheControl

VERSION = "1.2.0"


class FotMob:
    BASE_URL = "https://www.fotmob.com/api"
    LOGGER = getLogger(__name__)

    def __init__(
        self, proxies: Optional[dict] = None, logging_level: Optional[str] = "WARNING"
    ) -> None:
        SESSION = requests.Session()
        if proxies:
            SESSION.proxies.update(proxies)
        CACHE_SESSION = CacheControl(SESSION)

        if logging_level:
            if logging_level.upper() in [
                "DEBUG",
                "INFO",
                "WARNING",
                "ERROR",
                "CRITICAL",
            ]:
                self.LOGGER.setLevel(getLevelName(logging_level.upper()))
            else:
                print(f"Logging level {logging_level} not recognized!")

        self.session = CACHE_SESSION
        self.matches_url = f"{self.BASE_URL}/matches?"
        self.leagues_url = f"{self.BASE_URL}/leagues?"
        self.teams_url = f"{self.BASE_URL}/teams?"
        self.player_url = f"{self.BASE_URL}/playerData?"
        self.match_details_url = f"{self.BASE_URL}/matchDetails?"
        self.search_url = f"{self.BASE_URL}/searchData?"
        self.tv_listing_url = f"{self.BASE_URL}/tvlisting?"
        self.tv_listings_url = f"{self.BASE_URL}/tvlistings?"
        self.fixtures_url = f"{self.BASE_URL}/fixtures?"

    def _check_date(self, date: str) -> Union[re.Match, None]:
        pattern = re.compile(r"(20\d{2})(\d{2})(\d{2})")
        return pattern.match(date)

    def _execute_query(self, url: str):
        response = self.session.get(url)
        response.raise_for_status()
        self.LOGGER.debug(response)
        return response.json()

    def get_matches_by_date(
        self, date: str, time_zone: str = "America/New_York"
    ) -> dict:
        if self._check_date(date) != None:
            url = f"{self.matches_url}date={date}"
            return self._execute_query(url)
        return {}

    def get_league(
        self,
        id: int,
        tab: str = "overview",
        type: str = "league",
    ):
        url = f"{self.leagues_url}id={id}&tab={tab}&type={type}"
        return self._execute_query(url)

    def get_team(
        self,
        id: int,
        tab: str = "overview",
        type: str = "league",
        time_zone: str = "America/New_York",
    ):
        url = f"{self.teams_url}id={id}&tab={tab}&type={type}"
        return self._execute_query(url)

    def get_player(self, id: int):
        url = f"{self.player_url}id={id}"
        return self._execute_query(url)

    def get_match_details(self, match_id: int):
        url = f"{self.match_details_url}matchId={match_id}"
        return self._execute_query(url)

    def get_match_tv_listing(self, match_id: int, country_code: str = "GB"):
        url = f"{self.tv_listing_url}matchId={match_id}&countryCode={country_code}"
        return self._execute_query(url)

    def get_tv_listings_country(self, country_code: str = "GB"):
        url = f"{self.tv_listings_url}countryCode={country_code}"
        return self._execute_query(url)

    def search(self, term: str, user_language: str = "en-GB,en"):
        searchterm = urllib.parse.quote_plus(term)
        url = f"{self.search_url}term={searchterm}&userLanguage={user_language}"
        return self._execute_query(url)

    def get_fixtures(self, id: str, season: str):
        url = f"{self.fixtures_url}id={id}&season={season}%2F2024"
        print(url)
        return self._execute_query(url)

# data

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [5]:
Fot = FotMob()
f = Fot.get_fixtures("47", "2023")
# example data:
"""
[{'id': '4193450',
  'pageUrl': '/matches/burnley-vs-manchester-city/2ai7j8#4193450',
  'opponent': {'id': '8191', 'name': 'Burnley', 'score': 0},
  'home': {'id': '8191', 'name': 'Burnley', 'score': 0},
  'away': {'id': '8456', 'name': 'Man City', 'score': 3},
  'displayTournament': True,
  'lnameArr': ['A', ' | undefined'],
  'notStarted': False,
  'tournament': {},
  'status': {'utcTime': '2023-08-11T19:00:00.000Z',
   'finished': True,
   'started': True,
   'cancelled': False,
   'scoreStr': '0 - 3',
   'reason': {'short': 'FT',
    'shortKey': 'fulltime_short',
    'long': 'Full-Time',
    'longKey': 'finished'}}}]
"""

extracted_data = []
for match in f:
    if match["status"]["finished"]:
        extracted_data.append(
            {
                "home_score": match["home"]["score"],
                "away_score": match["away"]["score"],
                "home_team": match["home"]["name"],
                "away_team": match["away"]["name"],
            }
        )

data = pd.DataFrame(extracted_data)
data

https://www.fotmob.com/api/fixtures?id=47&season=2023%2F2024


,home_score,away_score,home_team,away_team
0,0,3,Burnley,Man City
1,2,1,Arsenal,Nottm Forest
2,1,1,Bournemouth,West Ham
3,4,1,Brighton,Luton
4,0,1,Everton,Fulham
...,...,...,...,...
116,1,1,Brighton,Sheff Utd
117,3,0,Liverpool,Brentford
118,3,2,West Ham,Nottm Forest
119,4,4,Chelsea,Man City


# main

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


In [7]:
selected_features = ["home_team", "away_team"]

label_encoder = LabelEncoder()
data["home_team"] = label_encoder.fit_transform(data["home_team"])
data["away_team"] = label_encoder.transform(data["away_team"])

X = data[selected_features]
y_home = data["home_score"]
y_away = data["away_score"]

(
    X_train,
    X_test,
    y_home_train,
    y_home_test,
    y_away_train,
    y_away_test,
) = train_test_split(X, y_home, y_away, test_size=0.2, random_state=42)

X_train_tensor = torch.from_numpy(X_train.astype(np.float32).values).to(device)
y_home_train_tensor = torch.from_numpy(y_home_train.astype(np.float32).values).to(device)
y_away_train_tensor = torch.from_numpy(y_away_train.astype(np.float32).values).to(device)
X_test_tensor = torch.from_numpy(X_test.astype(np.float32).values).to(device)
y_home_test_tensor = torch.from_numpy(y_home_test.astype(np.float32).values).to(device)
y_away_test_tensor = torch.from_numpy(y_away_test.astype(np.float32).values).to(device)

X

,home_team,away_team
0,5,12
1,0,15
2,2,18
3,4,11
4,8,9
...,...,...
116,4,16
117,10,3
118,18,15
119,6,12


In [21]:

class FootballScorePredictor(nn.Module):
    def __init__(self, input_size, team_vocab_size, embedding_dim, hidden_size, output_size):
        super(FootballScorePredictor, self).__init__()
        self.embedding_home = nn.Embedding(team_vocab_size, embedding_dim)
        self.embedding_away = nn.Embedding(team_vocab_size, embedding_dim)
        self.fc1 = nn.Linear(input_size + 2 * embedding_dim, hidden_size)
        self.relu = nn.ReLU()
        self.fc_home = nn.Linear(hidden_size, output_size)
        self.fc_away = nn.Linear(hidden_size, output_size)

    def forward(self, x, home_team, away_team):
        home_team = home_team.long()
        away_team = away_team.long()

        embed_home = self.embedding_home(home_team)
        embed_away = self.embedding_away(away_team)
        x = torch.cat((x, embed_home, embed_away), dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        home_score = self.fc_home(x)
        away_score = self.fc_away(x)
        return home_score, away_score

input_size = len(selected_features)
team_vocab_size = len(label_encoder.classes_)
embedding_dim = 10
hidden_size = 64
output_size = 1
model = FootballScorePredictor(input_size, team_vocab_size, embedding_dim, hidden_size, output_size)
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 100000
for epoch in range(num_epochs):

    home_outputs, away_outputs = model(X_train_tensor, X_train_tensor[:, -2], X_train_tensor[:, -1])
    loss_home = criterion(home_outputs.squeeze(), y_home_train_tensor)
    loss_away = criterion(away_outputs.squeeze(), y_away_train_tensor)
    loss = loss_home + loss_away


    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10000 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Total Loss: {loss.item()}')


Epoch [10000/100000], Total Loss: 5.053175217671624e-13
Epoch [20000/100000], Total Loss: 0.00011662362521747127
Epoch [30000/100000], Total Loss: 6.804944860050455e-05
Epoch [40000/100000], Total Loss: 1.5066767673488357e-06
Epoch [50000/100000], Total Loss: 4.573645128402859e-05
Epoch [60000/100000], Total Loss: 1.0856301742023788e-05
Epoch [70000/100000], Total Loss: 4.289830990433785e-10
Epoch [80000/100000], Total Loss: 0.00033551454544067383
Epoch [90000/100000], Total Loss: 0.00012523293844424188
Epoch [100000/100000], Total Loss: 1.3745078831561841e-05


### graph of test data

In [22]:
import plotly.express as px
import plotly.graph_objects as go


with torch.no_grad():
    home_outputs, away_outputs = model(X_test_tensor, X_test_tensor[:, -2], X_test_tensor[:, -1])  
    home_outputs = home_outputs.squeeze().cpu().numpy()  
    away_outputs = away_outputs.squeeze().cpu().numpy()  
    y_home_test_np = y_home_test_tensor.cpu().numpy()  
    y_away_test_np = y_away_test_tensor.cpu().numpy()  
    home_teams = label_encoder.inverse_transform(X_test['home_team'])  
    away_teams = label_encoder.inverse_transform(X_test['away_team'])  

    
    df_home = pd.DataFrame({
        'Team': home_teams,
        'Opponent': away_teams,
        'Actual Score': y_home_test_np,
        'Predicted Score': home_outputs
    })

    df_away = pd.DataFrame({
        'Team': away_teams,
        'Opponent': home_teams,
        'Actual Score': y_away_test_np,
        'Predicted Score': away_outputs
    })

    
    df_all = pd.concat([df_home, df_away])

    
    fig = px.scatter(df_all, x='Actual Score', y='Predicted Score', color='Team',
                     title='Teams Performance: Predicted vs Actual Scores',
                     labels={'Actual Score': 'Actual Score', 'Predicted Score': 'Predicted Score'},
                     hover_data=['Team', 'Opponent'])
    
    line_x_values = np.linspace(0, 6)
    line_y_values = line_x_values

    # Add a trace for the line y = x
    fig.add_trace(go.Scatter(x=line_x_values, y=line_y_values, mode='lines', name='y = x'))

    fig.update_traces(marker=dict(size=8, opacity=0.7), selector=dict(mode='markers'))
    fig.show()


### graph of training data

In [10]:
import plotly.express as px


with torch.no_grad():
    home_outputs, away_outputs = model(X_train_tensor, X_train_tensor[:, -2], X_train_tensor[:, -1])  
    home_outputs = home_outputs.squeeze().cpu().numpy()
    away_outputs = away_outputs.squeeze().cpu().numpy()  
    y_home_train_np = y_home_train_tensor.cpu().numpy()  
    y_away_train_np = y_away_train_tensor.cpu().numpy()  
    home_teams = label_encoder.inverse_transform(X_train['home_team'])  
    away_teams = label_encoder.inverse_transform(X_train['away_team'])  

    
    df_home = pd.DataFrame({
        'Team': home_teams,
        'Opponent': away_teams,
        'Actual Score': y_home_train_np,
        'Predicted Score': home_outputs
    })

    df_away = pd.DataFrame({
        'Team': away_teams,
        'Opponent': home_teams,
        'Actual Score': y_away_train_np,
        'Predicted Score': away_outputs
    })

    
    df_all = pd.concat([df_home, df_away])

    
    fig = px.scatter(df_all, x='Actual Score', y='Predicted Score', color='Team',
                     title='Teams Performance: Predicted vs Actual Scores',
                     labels={'Actual Score': 'Actual Score', 'Predicted Score': 'Predicted Score'},
                     hover_data=['Team', 'Opponent'])

    fig.update_traces(marker=dict(size=8, opacity=0.7), selector=dict(mode='markers'))
    fig.show()

# len(home_teams), len(away_teams), len(home_outputs), len(away_outputs)
# home_outputs

# using new data

In [11]:
extracted_data = []
for match in f:
    if not match["status"]["finished"]:
        extracted_data.append(
            {
                "home_team": match["home"]["name"],
                "away_team": match["away"]["name"],
            }
        )


new_data = pd.DataFrame(extracted_data)

In [12]:
selected_features = ["home_team", "away_team"]


label_encoder = LabelEncoder()
new_data["home_team"] = label_encoder.fit_transform(new_data["home_team"])
new_data["away_team"] = label_encoder.transform(new_data["away_team"])


X_new = new_data[selected_features]


X_new_tensor = torch.tensor(X_new.values, dtype=torch.float32)
X_new_tensor, X_new_tensor[:, -2], X_new_tensor[:, -1]

(tensor([[12., 10.],
         [ 5., 18.],
         [11.,  7.],
         [14.,  6.],
         [15.,  4.],
         [16.,  2.],
         [ 3.,  0.],
         [17.,  1.],
         [ 8., 13.],
         [ 9., 19.],
         [ 0., 19.],
         [ 3., 11.],
         [ 5., 16.],
         [15.,  8.],
         [14., 13.],
         [ 2.,  1.],
         [ 6.,  4.],
         [10.,  9.],
         [18.,  7.],
         [12., 17.],
         [19.,  5.],
         [11.,  0.],
         [ 4.,  3.],
         [ 7.,  2.],
         [ 9., 15.],
         [16., 10.],
         [ 1., 12.],
         [13.,  6.],
         [ 8., 14.],
         [17., 18.],
         [ 7., 10.],
         [ 4.,  5.],
         [13.,  2.],
         [16.,  3.],
         [19., 15.],
         [ 1.,  0.],
         [ 8.,  6.],
         [ 9., 18.],
         [11., 12.],
         [17., 14.],
         [15., 17.],
         [ 2., 11.],
         [ 6., 16.],
         [12.,  7.],
         [14.,  9.],
         [ 5.,  8.],
         [ 0.,  4.],
         [ 3.

In [13]:
with torch.no_grad():
    home_scores_pred, away_scores_pred = model(
        X_new_tensor, X_new_tensor[:, -2], X_new_tensor[:, -1]
    )

    home_scores_pred = home_scores_pred.squeeze().numpy()
    away_scores_pred = away_scores_pred.squeeze().numpy()

    home_teams = label_encoder.inverse_transform(new_data["home_team"])
    away_teams = label_encoder.inverse_transform(new_data["away_team"])

    df_home = pd.DataFrame(
        {
            "Team": home_teams,
            "Opponent": away_teams,
            "Predicted Score": home_scores_pred,
        }
    )

    df_away = pd.DataFrame(
        {
            "Team": away_teams,
            "Opponent": home_teams,
            "Predicted Score": away_scores_pred,
        }
    )

    df_all = pd.concat([df_home, df_away])

    fig = px.scatter(
        df_all,
        x="Team",
        y="Predicted Score",
        color="Team",
        title="Predicted Scores",
        hover_data=["Team", "Opponent"],
    )

    fig.update_traces(marker=dict(size=8, opacity=0.7), selector=dict(mode="markers"))
    fig.show()

    fig2 = px.scatter(
        df_all,
        x="Team",
        y="Opponent",
        color="Team",
        title="Predicted Scores, Team against Team",
        hover_data=["Team", "Opponent", "Predicted Score"],
    )

    fig2.update_traces(marker=dict(size=8, opacity=0.7), selector=dict(mode="markers"))
    fig2.show()